In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
! pip install -U langchain langchain-community langchain-chroma
! pip install -U langchain-google-genai
! pip install -U sentence-transformers chromadb
!pip install -U pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 3.2.0 requires google-ai-generativelanguage<1.0.0,>=0.9.0, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [9]:
import os
from glob import glob

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
# 1) Load PDF
pdf_paths = glob("/content/drive/MyDrive/NLP/RAG_final/Raw_data/*.pdf")
docs = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())

# 2) Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks = text_splitter.split_documents(docs)

# 3) Embedding
embedding = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding"
)

# 4) Chroma vectorstore
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_pdfs_langchain",
)

print("✅ Index xong")


['/content/drive/MyDrive/NLP/RAG_final/Raw_data/Nhung_dieu_can_biet_khi_hoc_tai_TDTU_2025.pdf', '/content/drive/MyDrive/NLP/RAG_final/Raw_data/Quy_che_tin_chi_he_lien_thong.pdf', '/content/drive/MyDrive/NLP/RAG_final/Raw_data/Trich_luoc_quy_che_tuyen_sinh_va_dao_tao_trinh_do_Tien_si_TDTU.pdf', '/content/drive/MyDrive/NLP/RAG_final/Raw_data/QD_1830_Quy_che_to_chuc_va_quan_ly_dao_tao_tuyen_sinh_2021.pdf']
✅ Index xong


In [ ]:
from google.colab import userdata
Gemini_API_key = userdata.get('Gemini_API_key')

# 2) LLM = Gemini (qua LangChain)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=Gemini_API_key
)

# 3) Load lại Chroma + embedding
embedding = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding"
)

vectordb = Chroma(
    persist_directory="/content/drive/MyDrive/NLP/RAG_final/Embedding/chroma_pdfs_langchain",
    embedding_function=embedding,
)

retriever = vectordb.as_retriever(
    search_kwargs={"k": 8}
)

# 4) Prompt RAG
RAG_PROMPT = """
Bạn là trợ lý trả lời câu hỏi dựa trên các quy chế, quy định của trường đại học.

Ngữ cảnh dưới đây được trích từ tài liệu gốc (có thể nhiều đoạn, nhiều điều, chương khác nhau):

--------------------
{context}
--------------------

Yêu cầu:
- Chỉ sử dụng thông tin trong ngữ cảnh trên để trả lời.
- Trả lời ngắn gọn, rõ ràng, bằng tiếng Việt.
- Nếu ngữ cảnh không chứa đủ thông tin, hãy nói rõ là không tìm thấy quy định hoặc không đủ thông tin.

Câu hỏi của người dùng:
{input}

Câu trả lời:
"""

prompt = ChatPromptTemplate.from_template(RAG_PROMPT)


In [ ]:
def rag_answer(query: str, top_k: int = 8):
    docs = retriever.invoke(query)

    # 2) Ghép context từ các document
    context = "\n\n---\n\n".join(d.page_content for d in docs)

    # 3) Build prompt value từ template + context + câu hỏi
    prompt_value = prompt.invoke({
        "context": context,
        "input": query,
    })

    # 4) Gọi LLM (Gemini) qua LangChain
    response = llm.invoke(prompt_value)

    # tuỳ version, response có thể là .content hoặc .text
    answer = getattr(response, "content", None) or getattr(response, "text", "")

    # 5) Chuẩn hoá metadata để debug giống style cũ
    used_chunks = []
    for i, d in enumerate(docs, start=1):
        meta = d.metadata or {}
        used_chunks.append({
            "chunk_id": i,
            "source": meta.get("source"),
            "page": meta.get("page"),
        })

    return {
        "answer": answer,
        "used_chunks": used_chunks,
    }

In [ ]:
query = "Chương trình đào tạo của đại học Tôn Đức Thắng được tính theo đơn vị gì ?"
result = rag_answer(query, top_k=8)

print("=== Câu trả lời ===")
print(result["answer"])

print("\n=== Các đoạn context đã dùng ===")
for c in result["used_chunks"]:
    print(c)


=== Câu trả lời ===
Chương trình đào tạo của Trường Đại học Tôn Đức Thắng được xây dựng theo đơn vị tín chỉ.

=== Các đoạn context đã dùng ===
{'chunk_id': 1, 'source': '/content/drive/MyDrive/NLP/RAG_final/Raw_data/QD_1830_Quy_che_to_chuc_va_quan_ly_dao_tao_tuyen_sinh_2021.pdf', 'page': 0}
{'chunk_id': 2, 'source': '/content/drive/MyDrive/NLP/RAG_final/Raw_data/Quy_che_tin_chi_he_lien_thong.pdf', 'page': 2}
{'chunk_id': 3, 'source': '/content/drive/MyDrive/NLP/RAG_final/Raw_data/QD_1830_Quy_che_to_chuc_va_quan_ly_dao_tao_tuyen_sinh_2021.pdf', 'page': 0}
{'chunk_id': 4, 'source': '/content/drive/MyDrive/NLP/RAG_final/Raw_data/Quy_che_tin_chi_he_lien_thong.pdf', 'page': 1}
{'chunk_id': 5, 'source': '/content/drive/MyDrive/NLP/RAG_final/Raw_data/QD_1830_Quy_che_to_chuc_va_quan_ly_dao_tao_tuyen_sinh_2021.pdf', 'page': 1}
{'chunk_id': 6, 'source': '/content/drive/MyDrive/NLP/RAG_final/Raw_data/Quy_che_tin_chi_he_lien_thong.pdf', 'page': 0}
{'chunk_id': 7, 'source': '/content/drive/MyDrive/